In [1]:
import pandas as pd
import numpy as np

# Importación de datos

In [2]:
df_country_codes = pd.read_csv('../data/raw/morticidad/country_codes.csv')

df_mort_1 = pd.read_csv('../data/raw/morticidad/MortIcd7.csv')
df_mort_2 = pd.read_csv('../data/raw/morticidad/Morticd8.csv')
df_mort_3 = pd.read_csv('../data/raw/morticidad/MortIcd9.csv')
df_mort_4 = pd.read_csv('../data/raw/morticidad/Morticd10_part1.csv')
df_mort_5 = pd.read_csv('../data/raw/morticidad/Morticd10_part2.csv')

C:\Users\ecaba\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\ecaba\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\ecaba\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (2,4) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df_mort_1.head()

,Country,Admin1,SubDiv,Year,List,Cause,Sex,Frmat,IM_Frmat,Deaths1,...,Deaths21,Deaths22,Deaths23,Deaths24,Deaths25,Deaths26,IM_Deaths1,IM_Deaths2,IM_Deaths3,IM_Deaths4
0,1125,NaN,NaN,1955,07A,A000,1,1,8,107822,...,2013.0,2727.0,4126.0,NaN,NaN,214,40543,NaN,NaN,NaN
1,1125,NaN,NaN,1955,07A,A000,2,1,8,105750,...,1745.0,3465.0,5963.0,NaN,NaN,82,39632,NaN,NaN,NaN
2,1125,NaN,NaN,1955,07A,A001,1,1,8,1302,...,8.0,6.0,2.0,NaN,NaN,1,6,NaN,NaN,NaN
3,1125,NaN,NaN,1955,07A,A001,2,1,8,739,...,2.0,1.0,2.0,NaN,NaN,0,4,NaN,NaN,NaN
4,1125,NaN,NaN,1955,07A,A002,1,1,8,69,...,0.0,0.0,0.0,NaN,NaN,0,7,NaN,NaN,NaN


## Preprocesamiento de datos

In [4]:
# Concatenamos todos los dataframes
df_total = pd.DataFrame(columns=df_mort_1.columns)

dataframes = [df_mort_1, df_mort_2, df_mort_3, df_mort_4, df_mort_5]

for dataframe in dataframes:
    df_total = pd.concat([df_total, dataframe], ignore_index=True)
    
df_total.shape

(5506984, 39)

### Selección de países

In [5]:
# Selección de países
lista_cod_paises = [4280, 4085, 4038, 4070, 2450, 2090, 2310, 2020, 2070, 3190, 3160, 3350, 2130, 4150, 2180,
                    3150, 4276, 3030, 2340, 4290]

In [6]:
df_filtrado = df_total[df_total['Country'].isin(lista_cod_paises)]
df_filtrado.shape

(1810369, 39)

### Selección de columnas

In [7]:
# Selección de columnas
columns_to_keep = ['Country', 'Year', 'Deaths1']
df_filtrado = df_filtrado[columns_to_keep]

# Reseteamos el índice
df_filtrado.reset_index(drop=True, inplace=True)
df_filtrado.head()

,Country,Year,Deaths1
0,2020,1966,91182
1,2020,1966,62711
2,2020,1966,1356
3,2020,1966,752
4,2020,1966,90


In [8]:
# Sumamos las muertes totales por país y año
df_comprimido = df_filtrado.groupby(['Country', 'Year']).sum().reset_index()
df_comprimido.head()

,Country,Year,Deaths1
0,2020,1966,307786
1,2020,1967,369282
2,2020,1968,426626
3,2020,1969,446410
4,2020,1970,444308


### Selección de años

In [9]:
# Selección de años
df_anios = df_comprimido[df_comprimido['Year'] > 1994]
df_country_codes.columns = ["Country","Name"]
df_anios = df_anios.merge(df_country_codes, how="inner", on='Country')

In [10]:
# Creación de un dataset con todos los años
df_final = pd.get_dummies(df_anios['Year'], prefix='Year')
df_final.drop(df_final.index, inplace=True)

In [11]:
# Comprobamos que los países elegidos tienen datos en los 23 años seleccionados
df_anios['Name'].value_counts().head(40)

Brunei Darussalam           23
Israel                      23
Finland                     23
Colombia                    23
Sweden                      23
Brazil                      23
Mexico                      23
Singapore                   23
Hungary                     23
Argentina                   23
Spain                       23
United States of America    23
Slovenia                    23
Germany                     23
Japan                       23
Nicaragua                   23
Ecuador                     23
Canada                      23
Croatia                     23
Kuwait                      23
Name: Name, dtype: int64

### Introducción de datos

In [12]:
# Introducimos los datos de cada país en el dataset
index = 0
for i in range(20):
    muertes_pais = df_anios[index:index+23]['Deaths1']
    muertes_pais = muertes_pais.to_frame().T
    muertes_pais.columns = df_final.columns[:-1]
    muertes_pais.index = [df_anios.iloc[index]['Name']]
    
    df_final = df_final.append(muertes_pais)
    
    index = index+23

df_final.drop('Year_2018', axis=1, inplace=True)
df_final.sort_index(inplace=True)

In [13]:
df_final

,Year_1995,Year_1996,Year_1997,Year_1998,Year_1999,Year_2000,Year_2001,Year_2002,Year_2003,Year_2004,...,Year_2008,Year_2009,Year_2010,Year_2011,Year_2012,Year_2013,Year_2014,Year_2015,Year_2016,Year_2017
Argentina,663472,664675,541820,560360,579086,554296,571882,582380,603740,587718,...,599354,603736,632596,633430,634288,646654,644070,661176,700234,683376
Brazil,4427790,3635528,3614064,3716092,3754632,3785564,3845928,3931176,2004624,2048138,...,2150282,2206176,2273892,2340996,2362332,2420948,2454078,2528350,2619548,2625328
Brunei Darussalam,2796,2525,2697,2582,2766,2895,2991,2898,2901,3097,...,3359,3440,2470,2432,2800,2940,3094,3264,3392,3224
Canada,653537,660572,669303,677740,681742,436124,439076,447206,452338,453168,...,477234,476836,480150,484148,493192,504676,517642,528666,534426,553378
Colombia,427210,437252,341064,350726,367106,374552,382744,384148,379600,377562,...,393496,393366,401044,391646,399512,406142,420102,438944,446156,455248
Croatia,101072,101272,103928,104622,103906,100492,99104,101138,105150,99512,...,104302,104828,104192,102038,103420,100772,101678,108410,103084,106954
Ecuador,131462,135887,104108,108564,111754,112670,110384,111062,106964,109394,...,119900,119258,123232,124608,127022,126208,125962,129580,135012,138494
Finland,143541,98322,98286,98474,98736,98632,97008,98778,98066,95514,...,98180,99808,101820,101136,103474,102956,104818,104604,107928,107340
Germany,2790733,2786353,2718199,1704764,1692660,1677594,1657082,1683372,1707892,1636542,...,1688878,1709088,1717536,1704656,1739164,1787650,1736712,1850400,1821801,1864544
Hungary,445155,286260,278868,281740,286420,271202,264366,265666,271646,264984,...,260054,260828,260912,257590,258880,253556,252616,263394,254106,263348


In [14]:
# Exportación del dataset
# df_final.to_csv('../data/clean/deaths_clean.csv')